In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
# set tf to cpu only
tf.config.set_visible_devices([], 'GPU')

import jax
jax.config.update('jax_platform_name', 'cpu')


from typing import Any
import numpy as np
import matplotlib.pyplot as plt 
import torch
from torch.utils.data import DataLoader

from mtr.config import cfg, cfg_from_yaml_file

from bc.behavior_cloning import BehaviorCloning
from bc.bc_dataset import BCDataset

from rl_env.env_utils import inverse_unicycle_control,smooth_scenario

torch.set_float32_matmul_precision('high')



In [ ]:
# fake dataset
from waymax import config as _config
from waymax import visualization
from rl_env.waymax_util import WomdLoader
from rl_env.env_utils import merge_dict, process_input
import mediapy

WOMD_1_2_0_VAL_LOCAL = _config.DatasetConfig(
    # path='/Data/Dataset/Waymo/V1_2_tf/validation_interactive/validation_interactive_tfexample.tfrecord@150',
    path='/Data/Dataset/Waymo/V1_2_tf/validation/validation_tfexample.tfrecord@150',
    # path='/Data/Dataset/Waymo/V1_2_tf/training/training_tfexample.tfrecord@1000',
    max_num_rg_points=30000,
    data_format=_config.DataFormat.TFRECORD,
    max_num_objects=32,
    shuffle_seed = 0,
)
data_iter = WomdLoader(data_config=WOMD_1_2_0_VAL_LOCAL)

# Test with Discrete

In [ ]:
cfg_file = 'tools/cfgs/waymo/bc_atten_discrete.yaml'
cfg_from_yaml_file(cfg_file, cfg)

model_path = 'output/bc_discrete_4_freeze/epoch=0-step=145000.ckpt'

BC = BehaviorCloning.load_from_checkpoint(model_path, map_location='cpu')

_ = BC.to('cuda')
_ = BC.eval()


BC.decoder.pred_all_layers = False

In [ ]:
scenario_id, scenario = data_iter.next()
img = visualization.plot_simulator_state(scenario, use_log_traj=True, highlight_obj = _config.ObjectType.MODELED)
mediapy.show_image(img)

In [ ]:
num_agent = len(scenario.object_metadata.is_sdc)
is_ctrl = np.zeros(num_agent, dtype=bool)
a = 0
is_ctrl[a] = True
t= 10

In [ ]:
print(t)
input = process_input(
    scenario,
    is_ctrl,
    from_gt=True,
    current_time_index=t,
    hide_history=11
    )

gt_action, action_valid = inverse_unicycle_control(scenario)

# gt_action = np.asarray(
# compute_inverse(scenario.log_trajectory, t, estimate_yaw_with_velocity=True).data[is_ctrl]
# )
input['gt_action'] = gt_action[a, t:t+1]
batch = merge_dict([input], 'cuda')

BC.zero_grad()
with torch.no_grad():
    encoder_output = BC.encoder(batch, retain_input=True)
    decoder_output = BC.decoder(encoder_output)
    total_loss_best, tb_dict_best = BC.decoder.get_loss(decoder_output, debug = True)

print('Sample', BC.decoder.sample(decoder_output, True))

print('best', total_loss_best)
print(tb_dict_best)

t += 1

# Test with Control



In [ ]:
cfg_file = 'tools/cfgs/waymo/bc_atten_ctrl.yaml'
cfg_from_yaml_file(cfg_file, cfg)

model_path = 'output/bc_bicycle_4_freeze/epoch=0-step=15000-v1.ckpt'

BC = BehaviorCloning.load_from_checkpoint(model_path, map_location='cpu')
_ = BC.to('cuda')
_ = BC.eval()

In [ ]:
scenario_id, scenario = data_iter.next()
img = visualization.plot_simulator_state(scenario, use_log_traj=True, highlight_obj = _config.ObjectType.MODELED)
mediapy.show_image(img)

In [ ]:
smoothed_scenario = smooth_scenario(scenario, duplicate=True)
gt_action, gt_valid = inverse_unicycle_control(smoothed_scenario)

In [ ]:
num_agent = len(scenario.object_metadata.is_sdc)
is_ctrl = np.zeros(num_agent, dtype=bool)
a = 1
is_ctrl[a] = True
t= 1

In [ ]:
input = process_input(
    smoothed_scenario,
    is_ctrl,
    from_gt=True,
    current_time_index=t,
    hide_history=1
)
input['gt_action'] = gt_action[a, t:t+1]
batch = merge_dict([input], 'cuda')

BC.zero_grad()
with torch.no_grad():
    encoder_output = BC.encoder(batch, retain_input=True)
    decoder_output = BC.decoder(encoder_output)
    total_loss_best, tb_dict_best = BC.decoder.get_loss_best(decoder_output, debug = True)
    # build GMM
    (pred_states, pred_scores) = decoder_output['pred_list'][-1]
    mode, mix, gmm = BC.decoder.build_gmm_distribution(pred_states, pred_scores)
    
A_scale = np.diag(BC.decoder.output_std.cpu().detach().numpy())
    
print('Sample', BC.decoder.sample(decoder_output, False))

print('best', total_loss_best)
print(tb_dict_best)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

weights = mix.probs[0].cpu().detach().numpy()
# weights = np.ones((6,))/6
means = mode.loc[0].cpu().detach().numpy()
covariances = mode.covariance_matrix[0].cpu().detach().numpy()

means = means@A_scale + BC.decoder.output_mean.cpu().detach().numpy()
covariances = A_scale@covariances@A_scale

# Generate a grid of points to evaluate the contour
x = np.linspace(-8, 8, 100)
y = np.linspace(-0.8, 0.8, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

# Evaluate the PDF (probability density function) of the SciPy GMM at each point
Z = np.zeros_like(X)
for i in range(len(weights)):
    Z += weights[i]*multivariate_normal.pdf(pos, mean=means[i], cov=covariances[i])
    # Z += multivariate_normal.pdf(pos, mean=means[i], cov=covariances[i])

# Plot the contour
CS = plt.contour(X, Y, np.log(Z), levels = np.linspace(-2, 5, 5), alpha = 0.5)
CSF = plt.contourf(X, Y, Z+1e-5, cmap = plt.cm.Blues)
# plt.gca().clabel(CS, CS.levels, inline=True, fontsize=10)
plt.scatter(gt_action[a, t, 0], gt_action[a, t, 1], c='r', s=30, marker='x')
# plt.colorbar(CSF)
plt.xlabel('Acceleration')
plt.ylabel('Yaw Rate')
plt.title(f'Contour Plot of GMM {t}')
plt.show()

t+=1